In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets, ensemble
import optuna
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm

from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
path = '/content/drive/MyDrive/DACON/lg_am/'

train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

In [ ]:
target_list = ['Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 'Y_06',
       'Y_07', 'Y_08', 'Y_09', 'Y_10', 'Y_11', 'Y_12', 'Y_13', 'Y_14']

In [ ]:
for i in tqdm(range(len(target_list))):
    globals()['train_{}'.format(target_list[i])] = train[['X_01', 'X_02', 'X_03', 'X_05', 'X_06', 'X_07', 'X_08',
       'X_09', 'X_10', 'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17',
       'X_18', 'X_19', 'X_20', 'X_21', 'X_22', 'X_24', 'X_25', 'X_26',
       'X_27', 'X_28', 'X_29', 'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35',
       'X_36', 'X_37', 'X_38', 'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44',
       'X_45', 'X_46', 'X_49', 'X_50', 'X_51', 'X_52', 'X_53',
       'X_54', 'X_55', 'X_56', f'{target_list[i]}']]
    # locals()['train_{}'.format(target_list[i])].drop(columns=['X_48', 'X_47', 'X_23', 'X_04'], inplace=True)

100%|██████████| 14/14 [00:00<00:00, 157.58it/s]


## Y_07

In [ ]:
feature = ['X_01', 'X_02', 'X_03', 'X_05', 'X_06', 'X_07', 'X_08', 'X_09', 'X_10',
       'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17', 'X_18', 'X_19',
       'X_20', 'X_21', 'X_22', 'X_24', 'X_25', 'X_26', 'X_27', 'X_28', 'X_29',
       'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35', 'X_36', 'X_37', 'X_38',
       'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44', 'X_45', 'X_46', 'X_49',
       'X_50', 'X_51', 'X_52', 'X_53', 'X_54', 'X_55']

In [ ]:
X = train_Y_07[feature]
y = train_Y_07.loc[:, 'Y_07':]

target = test[X.columns]
kf = KFold(n_splits = 5, random_state = 42, shuffle = True)

In [ ]:
def objective_cat(trial):
    """
    Objective function to tune a `CatBoostRegressor` model.
    """

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    params = {
        'criterion' :  'squared_error',
        'n_estimators':trial.suggest_int("n_estimators", 30, 400),
        'max_depth': trial.suggest_int('max_depth',1, 15),
        'min_samples_split': trial.suggest_int('min_samples_split',2,30),        
        'min_samples_leaf': trial.suggest_int('min_samples_leaf',1,30),    
        'min_weight_fraction_leaf' : trial.suggest_uniform('min_weight_fraction_leaf',0.0, 0.5),
        'bootstrap' : True,
        'oob_score' : True,
        'warm_start' : True,
        'verbose' : False
    }


    model = RandomForestRegressor(
        # objective = 'MultiRMSE',
        random_state=42,
        **params,
    )
    model.fit(x_train, y_train)

    pred = model.predict(x_test)

    rmse = mean_squared_error(y_test, pred, squared=False)


    return rmse

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective_cat, n_trials=30)

[I 2022-08-07 10:14:45,153] A new study created in memory with name: no-name-3a8ab81e-417a-4417-ad06-736288258380
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
[I 2022-08-07 10:15:05,376] Trial 0 finished with value: 0.41426135481237586 and parameters: {'n_estimators': 319, 'max_depth': 12, 'min_samples_split': 3, 'min_samples_leaf': 18, 'min_weight_fraction_leaf': 0.3062197855390119}. Best is trial 0 with value: 0.41426135481237586.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
[I 2022-08-07 10:15:45,196] Trial 1 finished with value: 0.4122040276513959 and parameters: {'n_estimators': 248, 'max_depth': 5, 'min_samples_split': 27, 'min_samples_

In [ ]:
params=study.best_params 
params['criterion'] = 'squared_error'
params['bootstrap'] = True
params['oob_score'] = True
params['warm_start'] = True
params['verbose'] = False

In [ ]:
NRMSES = []
cb_pred = np.zeros(target.shape[0])
for i, idx in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.loc[idx[0]], y.loc[idx[0]]
    val_x, val_y = X.loc[idx[1]], y.loc[idx[1]]
    
    cb = RandomForestRegressor(**params)
    cb.fit(tr_x, tr_y)
    
    val_pred = cb.predict(val_x)
    NRMSE = mean_squared_error(val_y, val_pred, squared=False)
    print(f"{i + 1} Fold RMSE = {NRMSE}")
    NRMSES.append(NRMSE)
    
    fold_pred = cb.predict(target) / kf.n_splits
    cb_pred += fold_pred

In [ ]:
np.mean(NRMSES)

0.40853091747676434

## Y_08

In [ ]:
feature = ['X_01', 'X_02', 'X_03', 'X_05', 'X_06', 'X_07', 'X_08', 'X_09', 'X_10',
       'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17', 'X_18', 'X_19',
       'X_20', 'X_21', 'X_22', 'X_24', 'X_25', 'X_26', 'X_27', 'X_28', 'X_29',
       'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35', 'X_36', 'X_37', 'X_38',
       'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44', 'X_45', 'X_46', 'X_49',
       'X_50', 'X_51', 'X_52', 'X_53', 'X_54', 'X_55']

In [ ]:
X = train_Y_08[feature]
y = train_Y_08.loc[:, 'Y_08':]

target = test[X.columns]
kf = KFold(n_splits = 5, random_state = 42, shuffle = True)

In [ ]:
def objective_cat(trial):
    """
    Objective function to tune a `CatBoostRegressor` model.
    """

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    params = {
        'criterion' :  'squared_error',
        'n_estimators':trial.suggest_int("n_estimators", 30, 400),
        'max_depth': trial.suggest_int('max_depth',1, 15),
        'min_samples_split': trial.suggest_int('min_samples_split',2,30),        
        'min_samples_leaf': trial.suggest_int('min_samples_leaf',1,30),    
        'min_weight_fraction_leaf' : trial.suggest_uniform('min_weight_fraction_leaf',0.0, 0.5),
        'bootstrap' : True,
        'oob_score' : True,
        'warm_start' : True,
        'verbose' : False
    }


    model = RandomForestRegressor(
        # objective = 'MultiRMSE',
        random_state=42,
        **params,
    )
    model.fit(x_train, y_train)

    pred = model.predict(x_test)

    rmse = mean_squared_error(y_test, pred, squared=False)


    return rmse

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective_cat, n_trials=30)

In [ ]:
params=study.best_params 
params['criterion'] = 'squared_error'
params['bootstrap'] = True
params['oob_score'] = True
params['warm_start'] = True
params['verbose'] = False

In [ ]:
NRMSES = []
cb_pred = np.zeros(target.shape[0])
for i, idx in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.loc[idx[0]], y.loc[idx[0]]
    val_x, val_y = X.loc[idx[1]], y.loc[idx[1]]
    
    cb = RandomForestRegressor(**params)
    cb.fit(tr_x, tr_y)
    
    val_pred = cb.predict(val_x)
    NRMSE = mean_squared_error(val_y, val_pred, squared=False)
    print(f"{i + 1} Fold RMSE = {NRMSE}")
    NRMSES.append(NRMSE)
    
    fold_pred = cb.predict(target) / kf.n_splits
    cb_pred += fold_pred

In [ ]:
np.mean(NRMSES)

0.6256107033342393

## Y_09

In [ ]:
feature = ['X_01', 'X_02', 'X_03', 'X_05', 'X_06', 'X_07', 'X_08', 'X_09', 'X_10',
       'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17', 'X_18', 'X_19',
       'X_20', 'X_21', 'X_22', 'X_24', 'X_25', 'X_26', 'X_27', 'X_28', 'X_29',
       'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35', 'X_36', 'X_37', 'X_38',
       'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44', 'X_45', 'X_46', 'X_49',
       'X_50', 'X_51', 'X_52', 'X_53', 'X_54', 'X_55']

In [ ]:
X = train_Y_09[feature]
y = train_Y_09.loc[:, 'Y_09':]

target = test[X.columns]
kf = KFold(n_splits = 5, random_state = 42, shuffle = True)

In [ ]:
def objective_cat(trial):
    """
    Objective function to tune a `CatBoostRegressor` model.
    """

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    params = {
        'criterion' :  'squared_error',
        'n_estimators':trial.suggest_int("n_estimators", 30, 400),
        'max_depth': trial.suggest_int('max_depth',1, 15),
        'min_samples_split': trial.suggest_int('min_samples_split',2,30),        
        'min_samples_leaf': trial.suggest_int('min_samples_leaf',1,30),    
        'min_weight_fraction_leaf' : trial.suggest_uniform('min_weight_fraction_leaf',0.0, 0.5),
        'bootstrap' : True,
        'oob_score' : True,
        'warm_start' : True,
        'verbose' : False
    }


    model = RandomForestRegressor(
        # objective = 'MultiRMSE',
        random_state=42,
        **params,
    )
    model.fit(x_train, y_train)

    pred = model.predict(x_test)

    rmse = mean_squared_error(y_test, pred, squared=False)


    return rmse

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective_cat, n_trials=30)

In [ ]:
params=study.best_params 
params['criterion'] = 'squared_error'
params['bootstrap'] = True
params['oob_score'] = True
params['warm_start'] = True
params['verbose'] = False

In [ ]:
NRMSES = []
cb_pred = np.zeros(target.shape[0])
for i, idx in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.loc[idx[0]], y.loc[idx[0]]
    val_x, val_y = X.loc[idx[1]], y.loc[idx[1]]
    
    cb = RandomForestRegressor(**params)
    cb.fit(tr_x, tr_y)
    
    val_pred = cb.predict(val_x)
    NRMSE = mean_squared_error(val_y, val_pred, squared=False)
    print(f"{i + 1} Fold RMSE = {NRMSE}")
    NRMSES.append(NRMSE)
    
    fold_pred = cb.predict(target) / kf.n_splits
    cb_pred += fold_pred

In [ ]:
np.mean(NRMSES)

0.6215526533050436

## Y_12

In [ ]:
feature = ['X_01', 'X_02', 'X_03', 'X_05', 'X_06', 'X_07', 'X_08', 'X_09', 'X_10',
       'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17', 'X_18', 'X_19',
       'X_20', 'X_21', 'X_22', 'X_24', 'X_25', 'X_26', 'X_27', 'X_28', 'X_29',
       'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35', 'X_36', 'X_37', 'X_38',
       'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44', 'X_45', 'X_46', 'X_49',
       'X_50', 'X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56']

In [ ]:
X = train_Y_12[feature]
y = train_Y_12.loc[:, 'Y_12':]

target = test[X.columns]
kf = KFold(n_splits = 5, random_state = 42, shuffle = True)

In [ ]:
def objective_cat(trial):
    """
    Objective function to tune a `CatBoostRegressor` model.
    """

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    params = {
        'criterion' :  'squared_error',
        'n_estimators':trial.suggest_int("n_estimators", 30, 400),
        'max_depth': trial.suggest_int('max_depth',1, 15),
        'min_samples_split': trial.suggest_int('min_samples_split',2,30),        
        'min_samples_leaf': trial.suggest_int('min_samples_leaf',1,30),    
        'min_weight_fraction_leaf' : trial.suggest_uniform('min_weight_fraction_leaf',0.0, 0.5),
        'bootstrap' : True,
        'oob_score' : True,
        'warm_start' : True,
        'verbose' : False
    }


    model = RandomForestRegressor(
        # objective = 'MultiRMSE',
        random_state=42,
        **params,
    )
    model.fit(x_train, y_train)

    pred = model.predict(x_test)

    rmse = mean_squared_error(y_test, pred, squared=False)


    return rmse

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective_cat, n_trials=30)

In [ ]:
params=study.best_params 
params['criterion'] = 'squared_error'
params['bootstrap'] = True
params['oob_score'] = True
params['warm_start'] = True
params['verbose'] = False

In [ ]:
NRMSES = []
cb_pred = np.zeros(target.shape[0])
for i, idx in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.loc[idx[0]], y.loc[idx[0]]
    val_x, val_y = X.loc[idx[1]], y.loc[idx[1]]
    
    cb = RandomForestRegressor(**params)
    cb.fit(tr_x, tr_y)
    
    val_pred = cb.predict(val_x)
    NRMSE = mean_squared_error(val_y, val_pred, squared=False)
    print(f"{i + 1} Fold RMSE = {NRMSE}")
    NRMSES.append(NRMSE)
    
    fold_pred = cb.predict(target) / kf.n_splits
    cb_pred += fold_pred

In [ ]:
np.mean(NRMSES)

0.6383728120753893

## Y_13

In [ ]:
feature = ['X_01', 'X_02', 'X_03', 'X_05', 'X_06', 'X_07', 'X_08', 'X_09', 'X_10',
       'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17', 'X_18', 'X_19',
       'X_20', 'X_21', 'X_22', 'X_24', 'X_25', 'X_26', 'X_27', 'X_28', 'X_29',
       'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35', 'X_36', 'X_37', 'X_38',
       'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44', 'X_45', 'X_46', 'X_49',
       'X_50', 'X_51', 'X_52', 'X_53', 'X_54', 'X_55']

In [ ]:
X = train_Y_13[feature]
y = train_Y_13.loc[:, 'Y_13':]

target = test[X.columns]
kf = KFold(n_splits = 5, random_state = 42, shuffle = True)

In [ ]:
def objective_cat(trial):
    """
    Objective function to tune a `CatBoostRegressor` model.
    """

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    params = {
        'criterion' :  'squared_error',
        'n_estimators':trial.suggest_int("n_estimators", 30, 400),
        'max_depth': trial.suggest_int('max_depth',1, 15),
        'min_samples_split': trial.suggest_int('min_samples_split',2,30),        
        'min_samples_leaf': trial.suggest_int('min_samples_leaf',1,30),    
        'min_weight_fraction_leaf' : trial.suggest_uniform('min_weight_fraction_leaf',0.0, 0.5),
        'bootstrap' : True,
        'oob_score' : True,
        'warm_start' : True,
        'verbose' : False
    }


    model = RandomForestRegressor(
        # objective = 'MultiRMSE',
        random_state=42,
        **params,
    )
    model.fit(x_train, y_train)

    pred = model.predict(x_test)

    rmse = mean_squared_error(y_test, pred, squared=False)


    return rmse

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective_cat, n_trials=30)

In [ ]:
params=study.best_params 
params['criterion'] = 'squared_error'
params['bootstrap'] = True
params['oob_score'] = True
params['warm_start'] = True
params['verbose'] = False

In [ ]:
NRMSES = []
cb_pred = np.zeros(target.shape[0])
for i, idx in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.loc[idx[0]], y.loc[idx[0]]
    val_x, val_y = X.loc[idx[1]], y.loc[idx[1]]
    
    cb = RandomForestRegressor(**params)
    cb.fit(tr_x, tr_y)
    
    val_pred = cb.predict(val_x)
    NRMSE = mean_squared_error(val_y, val_pred, squared=False)
    print(f"{i + 1} Fold RMSE = {NRMSE}")
    NRMSES.append(NRMSE)
    
    fold_pred = cb.predict(target) / kf.n_splits
    cb_pred += fold_pred

In [ ]:
np.mean(NRMSES)

0.6222298082558946

## Y_14

In [ ]:
feature = ['X_01', 'X_02', 'X_03', 'X_05', 'X_06', 'X_07', 'X_08', 'X_09', 'X_10',
       'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17', 'X_18', 'X_19',
       'X_20', 'X_21', 'X_22', 'X_24', 'X_25', 'X_26', 'X_27', 'X_28', 'X_29',
       'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35', 'X_36', 'X_37', 'X_38',
       'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44', 'X_45', 'X_46', 'X_49',
       'X_50', 'X_51', 'X_52', 'X_53', 'X_54', 'X_55']

In [ ]:
X = train_Y_14[feature]
y = train_Y_14.loc[:, 'Y_14':]

target = test[X.columns]
kf = KFold(n_splits = 5, random_state = 42, shuffle = True)

In [ ]:
def objective_cat(trial):
    """
    Objective function to tune a `CatBoostRegressor` model.
    """

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    params = {
        'criterion' :  'squared_error',
        'n_estimators':trial.suggest_int("n_estimators", 30, 400),
        'max_depth': trial.suggest_int('max_depth',1, 15),
        'min_samples_split': trial.suggest_int('min_samples_split',2,30),        
        'min_samples_leaf': trial.suggest_int('min_samples_leaf',1,30),    
        'min_weight_fraction_leaf' : trial.suggest_uniform('min_weight_fraction_leaf',0.0, 0.5),
        'bootstrap' : True,
        'oob_score' : True,
        'warm_start' : True,
        'verbose' : False
    }


    model = RandomForestRegressor(
        # objective = 'MultiRMSE',
        random_state=42,
        **params,
    )
    model.fit(x_train, y_train)

    pred = model.predict(x_test)

    rmse = mean_squared_error(y_test, pred, squared=False)


    return rmse

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective_cat, n_trials=30)

In [ ]:
params=study.best_params 
params['criterion'] = 'squared_error'
params['bootstrap'] = True
params['oob_score'] = True
params['warm_start'] = True
params['verbose'] = False

In [ ]:
NRMSES = []
cb_pred = np.zeros(target.shape[0])
for i, idx in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.loc[idx[0]], y.loc[idx[0]]
    val_x, val_y = X.loc[idx[1]], y.loc[idx[1]]
    
    cb = RandomForestRegressor(**params)
    cb.fit(tr_x, tr_y)
    
    val_pred = cb.predict(val_x)
    NRMSE = mean_squared_error(val_y, val_pred, squared=False)
    print(f"{i + 1} Fold RMSE = {NRMSE}")
    NRMSES.append(NRMSE)
    
    fold_pred = cb.predict(target) / kf.n_splits
    cb_pred += fold_pred

In [ ]:
np.mean(NRMSES)

0.6234742702235347